* Dan changes - writing code blindly
    0. Add more evaluation metrics. 
    1. use early stopping (based on validatio nset)
    2. For longer sequences than max_len - should truncate rather than dropping...
        * COmplications - for problems with local labelling we can't truncate, as we would lose the label! 
        For local (per position) labeling - we will extract a sliding window / chunks. For other problems (whole seqence), we'll simply truncate (start+end) I think!
    * put MAX_GLOBAL_SEQ_LEN as parameter instead of magic number
    
    
    * NOTE: In `build_model` - I believe input datatypes chould be changed to int16, instead of int32 and float 32. (Would save on memory per batch) - it's just the input encoding and input sequences encoding?. 
    
    
* The "target processing" code could be cleaned up a LOT by refactoring to expect a TUPLE :
    * `("name", whole sequence/local", "binary/multiclass/regression"). 
    
    
    
    * Nadav's transformer code - 
         * https://github.com/nadavbra/keras-seq-vec-transformer/blob/master/keras_seq_vec_transformer.py
         * (Huji CSE server - chaperone-02): `/cs/phd/nadavb/github_projects/keras_seq_vec_transformer/`

In [1]:
import os
import pickle
import numpy as np
import pandas as pd
import h5py
import matplotlib.pyplot as plt

import tensorflow as tf
# from tensorflow import keras 
from tensorflow import keras as keras
from IPython.display import display

# from pwas.shared_utils.util import log

from sklearn.metrics import matthews_corrcoef, r2_score, f1_score, precision_score, recall_score, balanced_accuracy_score, mean_absolute_error, mean_squared_error
from sklearn.metrics import classification_report, roc_auc_score, log_loss # DAN
from sklearn.model_selection import train_test_split

# from keras.callbacks import EarlyStopping 
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau


KERAS_SEQ_VEC_SCRIPT_PATH = "keras_seq_vec_transformer.py"
# ""/cs/phd/nadavb/github_projects/keras_seq_vec_transformer/keras_seq_vec_transformer.py" # ORIGINAL path in CSE server
with open(KERAS_SEQ_VEC_SCRIPT_PATH, 'r') as f:
    '''
    Some horrible hacks. To make keras_seq_vec_transformer work with tf.keras instead of just keras we run the following instead of
    just exec(f.read()).
    Note that also above we run from tensorflow import keras instead of just import keras
    '''
    import tensorflow.keras.backend as K 
    from tensorflow.keras.layers import LayerNormalization
    exec('\n'.join([line for line in f.read().splitlines() if not line.startswith('import keras') and not \
            line.startswith('from keras_layer_normalization')]))

In [2]:
ALL_AAS = 'ACDEFGHIKLMNPQRSTUVWXY'
N_ANNOTATIONS = 8943

# BENCHMARKS_DIR = '/cs/phd/nadavb/cafa_project/data/proteomic_benchmarks'
BENCHMARKS_DIR = '../data'

# PRETRAINED_MODEL_WEIGHTS_FILE_PATH = '/cs/phd/nadavb/cafa_project/data/model_weights/whole-donkey-13/epoch-48020.pkl'
# PRETRAINED_MODEL_WEIGHTS_FILE_PATH = '../models/old_model_weights_epoch_89930.pkl'
PRETRAINED_MODEL_WEIGHTS_FILE_PATH = '../models/epoch_28530_sample_91400000.pkl'

BENCHMARKS = [
#     'signalP_binary.dataset',
#     'scop.dataset',
#     'remote_homology',
#     'fluorescence', 
    'secondary_structure',
    
#     'disorder_secondary_structure',
## these were missing before - DAN :
#     "PhosphositePTM.dataset",  ### 9 min per epoch
#     "stability" 
             ]
  #### "phosphoserine.dataset",   ## sequence texts and label lengths are different - dataset needs to be changed in advance or dropped

MAX_GLOBAL_SEQ_LEN = 450 # 600 # set it here to make it easier to change. should be length used in training.  # DAN

MAX_EPOCHS=50 # max train epochs,
BATCH_SIZE = 16

FAST_RUN = True # False#

In [3]:
if FAST_RUN:
    MAX_EPOCHS = 1
    BATCH_SIZE = BATCH_SIZE//2 

In [4]:
## duplicate of code from "utils.py"
def sliding_truncate_df_seqs_lengthwise(row,max_length:int = MAX_GLOBAL_SEQ_LEN):
    "without sliding, just truncate  - based on: https://stackoverflow.com/questions/50962722/how-to-split-pandas-dataframe-single-row-into-two-rows"
    r_len = len(row)
    max_length -= 2
    if r_len > max_length:
        return(row[:max_length//2] + row[-max_length//2:]) # take first and last segments up to max length total
    return row


## https://stackoverflow.com/questions/18854620/whats-the-best-way-to-split-a-string-into-fixed-length-chunks-and-work-with-the
def chunkstring(string:str, length:int=MAX_GLOBAL_SEQ_LEN):
    length -= 2
    try: return list(string[0+i:length+i ] for i in range(0, len(string), length))
    except: 
        print("exception - string",string)
        print(type(string))

def chunk_df(df,length:int=MAX_GLOBAL_SEQ_LEN,cols=["text","labels"]):
    """
    Splits a df of strings and sequence labels into chunks of max length length. 
    Returns a new df, with instances that were "too long" now split into multiple rows.
    Should be used  for sequence labelling problems (i.e where we must keep all positions). 
    
    This function can also be applied (with modifications) to make "sliding windows" for LM/LANGUAGE MODEL training.
    
    Function rewritten due to error with duplicate indexes - now only support (for now) 2 columns
    """
    length -= 2
    df = pd.DataFrame({"text":df["text"].apply(chunkstring).explode().values,
                      "labels":df["labels"].astype(str).apply(chunkstring).explode().values},columns=cols)    
    return df


In [5]:
# # pd.read_csv("../data/TAPE_benchmarks/disorder_secondary_structure.valid.csv",nrows=2)
# pd.read_csv("../data/stability.train.csv",nrows=2)

In [6]:
# df = pd.read_csv("../data/neuropred.dataset.test.csv")
# df = pd.read_csv("../data/TAPE_benchmarks/disorder_secondary_structure.valid.csv")

df = pd.read_csv("../data/PhosphositePTM.dataset.train.csv")#,nrows=3)
# df = pd.read_csv("../data/phosphoserine.dataset.train.csv") # has different lengths  of text and labels

print("# rows",df.shape[0])
print("max len text",df["text"].str.len().max())
display(df["text"].str.len().describe())

diff_len = df["text"].str.len() - df["labels"].str.len()
assert max(diff_len)==0  # all rows should have same length of text and labels
df.tail()

# rows 43416
max len text 35213.0


count    43410.000000
mean       606.431537
std        671.138013
min          2.000000
25%        280.000000
50%        449.000000
75%        733.000000
max      35213.000000
Name: text, dtype: float64

,text,labels
43411,MDKEKDGIPISSLREITLLLRLRHPNIVELKEVVVGNHLESIFLVM...,0000000000000000000000000000000000000000000000...
43412,MYCLLAAPLCLLSLLLSPLSPAAPISPSEPIGQAYSLALYMQKNTS...,0000000000000000000000000000000000000000000000...
43413,MDLEAARNGTARRLDGDFELGSISNQSREKKKKVNLIGPLTLFRYS...,0000000000000000000000000000000000000000000000...
43414,MSLVLLSLAALCRSAVPREPTVQCGSETGPSPEWMLQHDLIPGDLR...,0000000000000000000000000000000000000000000000...
43415,MSVHYTLNLRVFWPLVTGLCTALVCLYHVLRGSGGARAEPADGVDG...,0000000000000000000000000000000000000000000000...


In [9]:
ALL_AAS = 'ACDEFGHIKLMNPQRSTUVWXY'
ADDITIONAL_TOKENS = ['<OTHER>', '<START>', '<END>', '<PAD>']

# Each sequence is added <START> and <END> tokens
ADDED_TOKENS_PER_SEQ = 2

n_aas = len(ALL_AAS)
aa_to_token_index = {aa: i for i, aa in enumerate(ALL_AAS)}
additional_token_to_index = {token: i + len(ALL_AAS) for i, token in enumerate(ADDITIONAL_TOKENS)}
token_to_index = {**aa_to_token_index, **additional_token_to_index}
index_to_token = {index: token for token, index in token_to_index.items()}
n_tokens = len(token_to_index)

def tokenize_seq(seq):
    other_token_index = additional_token_to_index['<OTHER>']
    return [additional_token_to_index['<START>']] + [aa_to_token_index.get(aa, other_token_index) for aa in seq] + \
            [additional_token_to_index['<END>']]

def tokenize_seqs(seqs,max_seq_len=MAX_GLOBAL_SEQ_LEN):
    
    tokenized_seqs = additional_token_to_index['<PAD>'] * np.ones((len(seqs), max_seq_len))
    
    for i, seq in enumerate(seqs):
        tokenized_seq = tokenize_seq(seq)
        assert len(tokenized_seq) <= max_seq_len
        tokenized_seqs[i, :len(tokenized_seq)] = tokenized_seq
        
    return tokenized_seqs

def create_model(max_seq_len):
    
    input_seq_layer = keras.layers.Input(shape = (max_seq_len,), dtype = np.int32, name = 'input-seq')
    input_annoatations_layer = keras.layers.Input(shape = (N_ANNOTATIONS,), dtype = np.float32, name = 'input-annotations')
    output_seq_layer, output_annoatations_layer = TransformerAutoEncoder(vocab_size = n_tokens, d_vec = N_ANNOTATIONS, \
            output_vec_activation = 'sigmoid', name = 'auto-encoder')([input_seq_layer, input_annoatations_layer])

    # An ugly hack meant to achieve nothing than renaming those layers.
    output_seq_layer = keras.layers.Reshape(output_seq_layer.shape[1:], name = 'output_seq_layer')(output_seq_layer)
    output_annoatations_layer = keras.layers.Reshape(output_annoatations_layer.shape[1:], name = 'output_annoatations_layer')\
            (output_annoatations_layer)

    return keras.models.Model(inputs = [input_seq_layer, input_annoatations_layer], outputs = [output_seq_layer, output_annoatations_layer])

def load_model_weights(model, path):
    with open(path, 'rb') as f:
        model_weights, optimizer_weights = pickle.load(f)
        model.set_weights(model_weights)
        

In [10]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

* The same processing needs to be applied to the validation set , as the train set.
* A few datasets lack a validation set. We can set them to have one in advance, or just randomly sample the train set and split it here. 

* NOTE - sequence length truncation is applied to labels as well, for per position data. This may be problematic for evaluation!!  - We may need to split into more segments for sake of evaluation..


In [11]:
def evaluate(X, raw_y_true):     
    Y_pred = model.predict(X) # array of probabilities
    Y_pred_classes = Y_pred.argmax(axis = -1) # try to get classes, not probability per class
#     print("y_pred",Y_pred)
#     print("Y_pred_classes",Y_pred_classes)

    ### some sklearn metrics will fail, since we are doing multiclass and regression etc'. But i'll just "Try" each # DAN
#     print("trying sklearn metrics")
    try:
        print("classification Report\n")
        print(classification_report(raw_y_true,Y_pred_classes))
    except:pass    
    try:
        print("MCC %.4f%" % matthews_corrcoef(raw_y_true,Y_pred_classes))
    except:pass
    try:
        print("F1 - macro avg %.4f%" % f1_score(raw_y_true,Y_pred_classes, average='macro'))
        print("precision - micro avg %.2f%%" % (100 * precision_score(raw_y_true,Y_pred_classes, average='micro')))
        print("Recall - macro avg %.2f%%" % (100 * recall_score(raw_y_true,Y_pred_classes, average='micro')))
              
        print("balanced_accuracy_score %.4f%" % balanced_accuracy_score(raw_y_true,Y_pred_classes))
    except:pass
    
    try:
        print("r2 %.4f%" % r2_score(raw_y_true,Y_pred.flatten())) # doesn't work? DAN
        print("mean_absolute_error %.4f%" % mean_absolute_error(raw_y_true,Y_pred.flatten()))
    except: pass
    
    try: #get probability if supported, then calc logloss, auc
#         Y_pred_proba = model.predict(test_X)[:,1] # this function is deprecated..
        print("roc_auc_score %.4f%" % roc_auc_score(raw_y_true,Y_pred[:,1]))
        print("log_loss %.4f%" % log_loss(raw_y_true,Y_pred[:,1]))
    except:pass
    
    if is_y_numeric:
        results = pd.DataFrame({'true': raw_y_true, 'pred': Y_pred.flatten()})
        print("spearman's rho (correlation)",results.corr(method="spearman"))
        print('R^2 score: %.2g' % r2_score(results['true'], results['pred']))
        
        print("mean absolute error score %.4g" % mean_absolute_error(results['true'], results['pred']))      
              
    else:
        if is_y_seq:
            results_true = []
            results_pred = []

            for true_seq, pred_seq in zip(raw_y_true, Y_pred.argmax(axis = -1)):
                for true_token, pred_token_index in zip(true_seq, pred_seq):
                    results_true.append(true_token)
                    results_pred.append('<PAD>' if pred_token_index == n_labels else unique_labels[pred_token_index])

            results = pd.DataFrame({'true': results_true, 'pred': results_pred})
        else:
            predicted_labels = [unique_labels[i] for i in Y_pred.argmax(axis = -1)]
            results = pd.DataFrame({'true': raw_y_true, 'pred': predicted_labels})

        confusion_matrix = results.groupby(['true', 'pred']).size().unstack().fillna(0)
        #### display confusion matrix
        if len(set(unique_labels))<20:
            print('Confusion matrix:')
            display(confusion_matrix)

        accuracy = (results['true'] == results['pred']).mean()
        imbalance = (results['true'].value_counts().max() / len(results))
        print('Accuracy: %.2f%%' % (100 * accuracy))
        print('Imbalance (most common label): %.2f%%' % (100 * imbalance))

#         if set(unique_labels) == {0, 1}:
        if len(set(unique_labels)) == 2:
            y_true = results['true'].astype(float)
            y_pred = results['pred'].astype(float)
            print('MCC: %.2f%%' % (100 * matthews_corrcoef(y_true, y_pred)))
            
            print("F1 - macro avg %.2f%%" % (100 * f1_score(y_true, y_pred, average='macro')))       
            print("precision - micro avg %.2f%%" % (100 * precision_score(y_true, y_pred, average='micro')))
            print("Recall - macro avg %.2f%%" % (100 * recall_score(y_true, y_pred, average='micro')))
   

* Code comment: It would be great to __refactor__ this - it is currently cumbersome and not productionable. 
    * It should be rewritten to expect a train/valid/test set and Ys, (+- the target type, or a function for each type of target) and the Y preprocessing to be a seperate, user defined step, _not trying to infer it here as part of the loop_. 
    
    
    
Data augmentation idea/experiment: 
    * Duplicate train subset, with sequence reversed (and also reverse y/labels if target is sequence_label).

In [12]:
for benchmark_name in BENCHMARKS:
    print('========== %s ==========' % benchmark_name)
    
    text_file_path = os.path.join(BENCHMARKS_DIR, '%s.benchmark.txt' % benchmark_name)
    
    train_set_file_path = os.path.join(BENCHMARKS_DIR, '%s.train.csv' % benchmark_name)
    valid_set_file_path = os.path.join(BENCHMARKS_DIR, '%s.valid.csv' % benchmark_name)
    test_set_file_path = os.path.join(BENCHMARKS_DIR, '%s.test.csv' % benchmark_name)
    
    ### printout of text from benchmarks text files/descriptions: 
    """
#     if os.path.exists(text_file_path):
#         with open(text_file_path, 'r') as f:
#             for line in f:
#                 print('[$$$] ' + line, end = '')
    """
    
    print('\n')
    ##### As originally used, the "Test" set was actually the validation set and vice versa. I leave the names unchanged to avoid bugs, but it is confusing!! Dan
    train_set = pd.read_csv(train_set_file_path).dropna().drop_duplicates()

    is_y_numeric = np.issubdtype(train_set['labels'].dtype, np.floating)   
    
    try: 
        valid_set = pd.read_csv(valid_set_file_path).dropna().drop_duplicates()
    except:
#     if valid_set is None: 
        print(f"validation set {valid_set_file_path} missing")
        print("splitting train to train and random validation set")
        ## we could do stratified sampling here, but requires checking if problem is seq based or not
        try: ## could also try based on checking is_y_numeric & categoricals
            train_set, valid_set = train_test_split(train_set,
                                                    stratify=train_set['labels'],test_size=0.1,random_state=42)
            print("Stratified sampling of validation set")
        except:
            print("randomly sampling validation set")
            train_set, valid_set = train_test_split(train_set,test_size=0.1,random_state=42)
    
    try: 
        test_set = pd.read_csv(test_set_file_path).dropna().drop_duplicates()
    except:
        print(f"test set {test_set} missing")
    
    if FAST_RUN:
        if "remote_homology" in benchmark_name:
#            "remote_homology special fast sampling" - over 1K classes
#             train_set = train_set.sample(frac=0.8,random_state=42) # a small sample will give errors in remote homology/massively multiclass
            train_set = pd.concat([train_set.sample(frac=0.1,random_state=42),train_set.drop_duplicates('labels')]).drop_duplicates()
        else:
            if ((train_set['labels'].nunique()<1200) and (not is_y_numeric)):
                train_set,_ = train_test_split(train_set,stratify=train_set['labels'],train_size=0.05,random_state=42) # stratified sampling 0 for multiclass
            else:
                train_set = train_set.sample(frac=0.1,random_state=42)
        valid_set = valid_set.sample(frac=0.01)
        test_set = test_set.sample(frac=0.01)

    
    train_set_size = len(train_set)
    test_set_size = len(test_set)
    valid_set_size =  len(valid_set)
    
    print(f'{train_set_size} training-set records, {valid_set_size} valid-set records, {test_set_size} test-set records')
      
    '''
    Determining which of the following y is:
    1. Numeric (could be either probabalistic (i.e. in the range 0-1) or not)
    2. Sequence
    3. Categorical
    '''
    if is_y_numeric:
        is_y_seq = False
        is_y_probability = train_set['labels'].min() >= 0 and train_set['labels'].max() <= 1
        print('Numeric (%sprobabilistic) label' % ('' if is_y_probability else 'not '))
    else: 
        ## we already know y isn't a number / "1.76322"
#         is_y_seq = train_set['labels'].apply(lambda value: isinstance(value, str)).all() and \
#                 (train_set['text'].str.len() == train_set['labels'].str.len()).all() ###ORIG
        is_y_seq = train_set['labels'].astype(str).str.len().max()>15 ### Dan hack - phosphosite (0/1s seq) wasn't being identified
    
        print("is_y_seq",is_y_seq)
        if is_y_seq:
            train_set['labels'] = train_set['labels'].astype(str)## add "astype(str)"
            unique_labels = sorted(set.union(*train_set['labels'].apply(set))) 
        else:
            unique_labels = sorted(train_set['labels'].unique())
                    
        n_labels = len(unique_labels)
        label_to_index = {label: i for i, label in enumerate(unique_labels)}
        
        if is_y_seq:
            print('Sequence output with %d tokens.' % n_labels)
        else:
            print('Categorical output with %d labels.' % n_labels)
    ################
    #### sequence/label length truncation or splitting done here, depending on if y is sequence
    if is_y_seq == False:
        train_set["text"] = train_set["text"].apply(sliding_truncate_df_seqs_lengthwise)
        test_set["text"] = test_set["text"].apply(sliding_truncate_df_seqs_lengthwise)
        valid_set["text"] = valid_set["text"].apply(sliding_truncate_df_seqs_lengthwise)
    else:
        train_set = chunk_df(train_set)      
        test_set = chunk_df(test_set)
        valid_set = chunk_df(valid_set)
        print("chunked train set shape:",train_set.shape)
#         print("chunked valid_set size:",valid_set.shape[0])
        print("chunked test_set size:",test_set.shape[0])
        
        train_set_size = len(train_set)
        test_set_size = len(test_set)
        valid_set_size =  len(valid_set)
    
    ### update max sequence length:
    # Adding +2 for the <START> and <END> tokens.
    max_seq_len = max(train_set['text'].str.len().max(), test_set['text'].str.len().max(), \
                      valid_set['text'].str.len().max()) + 2
    
    print(f'Max seq len: {max_seq_len}')
    if max_seq_len > MAX_GLOBAL_SEQ_LEN:
        max_seq_len = MAX_GLOBAL_SEQ_LEN
    
    ####################
    
    model = create_model(max_seq_len)
    load_model_weights(model, PRETRAINED_MODEL_WEIGHTS_FILE_PATH)
    input_seq_layer, input_annoatations_layer = model.input
    output_seq_layer, output_annoatations_layer = model.output
    
    if is_y_numeric:
        output_layer = keras.layers.Dense(1, activation = ('sigmoid' if is_y_probability else None))(output_annoatations_layer)
        loss = 'binary_crossentropy' if is_y_probability else 'mse'
    elif is_y_seq:
        # Adding another token for padding, making it (n_labels + 1) tokens. ## DAN - why would output need a padding token? why n+1 ???
        output_layer = keras.layers.Dense(n_labels + 1, activation = 'softmax')(output_seq_layer)
        loss = 'categorical_crossentropy'
    else:
        output_layer = keras.layers.Dense(n_labels, activation = 'softmax')(output_annoatations_layer)
        loss = 'categorical_crossentropy'
    
    model = keras.models.Model(inputs = [input_seq_layer, input_annoatations_layer], outputs = output_layer)
    model.compile(optimizer = keras.optimizers.Adam(lr = 1e-04), loss = loss) # DAN - changed to 2 instead of 1. default in literature is 3e-4
    
    train_X = [
        tokenize_seqs(train_set['text'].values, max_seq_len).astype(np.int32),
        np.zeros((train_set_size, N_ANNOTATIONS), dtype = np.int8)
    ]
    
    valid_X = [
        tokenize_seqs(valid_set['text'].values, max_seq_len).astype(np.int32),
        np.zeros((valid_set_size, N_ANNOTATIONS), dtype = np.int8)
    ]
    
    test_X = [
        tokenize_seqs(test_set['text'].values, max_seq_len).astype(np.int32),
        np.zeros((test_set_size, N_ANNOTATIONS), dtype = np.int8)
    ]
    
    ### I added code to apply the transformations to the validset - hopefully without bugs... DAN ? 
    if is_y_numeric:
        train_Y = train_set['labels'].values
        valid_Y = valid_set['labels'].values
    elif is_y_seq:
        
        train_Y = np.zeros((train_set_size, max_seq_len, n_labels + 1), dtype = np.int8) ## DAN - try +1 from max_seq_len due to padding
        valid_Y = np.zeros((valid_set_size, max_seq_len, n_labels + 1), dtype = np.int8) ## DAN - try +1 from max_seq_len due to padding
# valid_Y[i, j, label_to_index[token]] = 1        
        
        for i, output_seq in enumerate(train_set['labels']):
            for j, token in enumerate(output_seq):
                train_Y[i, j, label_to_index[token]] = 1
            # Add a pad token for the rest of the output.
            train_Y[i, np.arange(len(output_seq), max_seq_len), n_labels] = 1
        ## duplicate above, for validation set. Unsure if I could just add in a row of "valid_y above?? DAN  
        for i, output_seq in enumerate(valid_set['labels']):           
            for j, token in enumerate(output_seq):
                valid_Y[i, j, label_to_index[token]] = 1
            # Add a pad token for the rest of the output.
            valid_Y[i, np.arange(len(output_seq), max_seq_len), n_labels] = 1
    else:
        train_Y = np.zeros((train_set_size, n_labels), dtype = np.int8)
        train_Y[np.arange(train_set_size),[label_to_index[label] for label in train_set['labels']]] = 1
        
        valid_Y = np.zeros((valid_set_size, n_labels), dtype = np.int8)
        valid_Y[np.arange(valid_set_size),[label_to_index[label] for label in valid_set['labels']]] = 1
        
#########
    ### Train model, with early stopping on validation set
    
    model.fit(train_X, train_Y,
              batch_size = BATCH_SIZE,
              validation_data=(valid_X,valid_Y),
              callbacks = [ReduceLROnPlateau(patience=2,factor=0.35), EarlyStopping(patience=4)],
              epochs = MAX_EPOCHS,
              validation_batch_size=BATCH_SIZE,
              verbose=2)
    #### DAN: NOTE - keras already keeps evaluation, train error data..
    print('\n*** Training-set performance: ***')
    evaluate(train_X, train_set['labels'].values)
    print('*** validation performance: ***')
    evaluate(valid_X, valid_set['labels'].values)
    print('*** Test-set performance: ***')
    evaluate(test_X, test_set['labels'].values)
    
    print('\n' * 3)

========== secondary_structure ==========


868 training-set records, 22 valid-set records, 4 test-set records
is_y_seq True
Sequence output with 3 tokens.
chunked train set shape: (954, 2)
chunked test_set size: 4
Max seq len: 450
120/120 - 17s - loss: 1.3766 - val_loss: 1.3657 - lr: 1.0000e-04

*** Training-set performance: ***
classification Report

Confusion matrix:


pred,0,1,2,<PAD>
true,,,,
0,28819,17949,867,30229
1,20633,8697,772,14918
2,38193,18507,1657,34107


Accuracy: 18.19%
Imbalance (most common label): 42.94%
*** validation performance: ***
classification Report

Confusion matrix:


pred,0,1,2,<PAD>
true,,,,
0,822,490,12,819
1,447,207,13,328
2,1009,490,47,927


Accuracy: 19.18%
Imbalance (most common label): 44.07%
*** Test-set performance: ***
classification Report

Confusion matrix:


pred,0,1,2,<PAD>
true,,,,
0,86,56,2,74
1,82,46,7,92
2,156,88,13,161


Accuracy: 16.80%
Imbalance (most common label): 48.44%






In [13]:
Y_pred = model.predict(train_X)
# Y_pred = model.predict(test_X).argmax(-1)
Y_pred

array([[[0.27362844, 0.19358492, 0.25285643, 0.2799302 ],
        [0.19041027, 0.26470122, 0.27283078, 0.2720577 ],
        [0.29056835, 0.17761472, 0.27154985, 0.26026708],
        ...,
        [0.22333321, 0.19653027, 0.3090035 , 0.2711331 ],
        [0.22333321, 0.19653027, 0.3090035 , 0.2711331 ],
        [0.22333321, 0.19653027, 0.3090035 , 0.2711331 ]],

       [[0.27362666, 0.1935881 , 0.2528556 , 0.2799296 ],
        [0.19044796, 0.2646827 , 0.2728122 , 0.27205712],
        [0.24825507, 0.24139766, 0.22144781, 0.28889942],
        ...,
        [0.23692396, 0.28960449, 0.19760235, 0.27586916],
        [0.20648701, 0.26560277, 0.2574372 , 0.270473  ],
        [0.18884726, 0.23590352, 0.22572857, 0.34952065]],

       [[0.27359593, 0.19363168, 0.25284764, 0.27992472],
        [0.20930295, 0.2546454 , 0.2584631 , 0.27758855],
        [0.21482061, 0.24689646, 0.23582028, 0.3024626 ],
        ...,
        [0.23638913, 0.3140212 , 0.18939406, 0.26019555],
        [0.22478743, 0.227545